In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint, uniform
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# SE Block：引入 external_var 融合进注意力机制
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16, external_dim=0):
        super(SEBlock, self).__init__()
        self.external_dim = external_dim
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(in_channels + external_dim, in_channels // reduction_ratio)
        self.fc2 = nn.Linear(in_channels // reduction_ratio, in_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, external_var=None):
        batch_size, channels, _, _ = x.size()
        squeeze = self.global_avg_pool(x).view(batch_size, channels)
        
        if self.external_dim > 0 and external_var is not None:
            squeeze = torch.cat((squeeze, external_var), dim=1)
        
        excitation = F.relu(self.fc1(squeeze))
        excitation = self.sigmoid(self.fc2(excitation)).view(batch_size, channels, 1, 1)
        return x * excitation


# BottleNeck 模块：支持传递 external_var
class BottleNeck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, reduction_ratio=16, use_se=True, external_dim=0):
        super(BottleNeck, self).__init__()
        mid_channels = out_channels // 4

        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.se = SEBlock(out_channels, reduction_ratio, external_dim) if use_se else nn.Identity()
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x, external_var=None):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = self.se(out, external_var)
        out += self.shortcut(x)
        return F.relu(out)


# 主 SE-ResNet 网络结构
class SEResNetRS(nn.Module):
    def __init__(self, num_blocks=[2, 2, 2, 2], num_classes=1, external_dim=8):
        super(SEResNetRS, self).__init__()
        self.in_channels = 64
        self.external_dim = external_dim

        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1, bias=False)  # 输入通道为10
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(64, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(64, 64, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(64, 64, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(64, 64, num_blocks[3], stride=2)

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc1 = nn.Linear(64 + external_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(BottleNeck(in_channels, out_channels, stride, external_dim=self.external_dim))
        for _ in range(1, num_blocks):
            layers.append(BottleNeck(out_channels, out_channels, stride=1, external_dim=self.external_dim))
        return nn.ModuleList(layers)

    def forward_layer(self, layer_blocks, x, external_var):
        for block in layer_blocks:
            x = block(x, external_var)
        return x

    def forward(self, x, external_var):
        x = x.permute(0, 3, 1, 2)  # 转为 [batch, channel=10, H=30, W=12]
        x = F.relu(self.bn1(self.conv1(x)))

        x = self.forward_layer(self.layer1, x, external_var)
        x = self.forward_layer(self.layer2, x, external_var)
        x = self.forward_layer(self.layer3, x, external_var)
        x = self.forward_layer(self.layer4, x, external_var)

        x = self.global_pool(x).view(x.size(0), -1)
        combined = torch.cat((x, external_var), dim=1)
        
        x = F.relu(self.fc1(combined))
        x = F.relu(self.fc2(x))
        out = self.fc3(x)
        return out
